In [21]:
import math
import numpy as np

class OptionsBlackScholes:
    def __init__(self, strike: float, rate: float, 
                 tau: float, sigma: float, bounds: tuple, 
                 put_call: str):

        self.strike   = strike
        self.rate     = rate
        self.tau      = tau
        self.sigma    = sigma
        self.put_call = put_call
        self.x_low    = bounds[0]
        self.x_up     = bounds[1]

    def payoff(self, spot: float):
        if self.put_call == 'put':
            return max(self.strike - spot, 0)
        else:
            return max(spot - self.strike, 0)
    
    def get_coefficients(self, t: float, spot: float): 
        """Return coefficients of Black Scholes formula"""
        coeff = {'a': - 0.5 * (spot*self.sigma(t, spot))**2,
                 'b': - self.rate * spot,
                 'c': self.rate,
                 'd': 0}

        return coeff

    def adjust_grid(self, grid: np.array, dx: int):
        return grid

    def bound_cond_tup(self,  spot):
        if self.put_call = 'call':
            return max(spot - self.strike, 0)
        else:
            return max(self.strike - spot, 0)

    def bound_cond_x_low(self, t):
        if self.put_call == 'call':
            return 0.0
        else:
            return self.strike * np.exp(-self.rate * (self.tau - t))
    
    def bound_cond_x_up(self, t):
        if self.put_call == 'call':
            return self.strike * np.exp(-self.rate * (self.tau - t))
        else:
            return 0.0


class Barrier(OptionsBlackScholes):
    def __init__(self, rate, sigma, strike, barrier, t_up, x_low, x_up, knock = 'up-out', put_call = "call"):
        """ Constructor of a up-and-out European Call option
        Parameters:
            rate   : underlying asset's risk free interest rate
            sigma  : lambda function of underlying asset's standard deviation
            strike : opption's strike price
            barrier: level of the barrier
            t_up   : option's time to expriry
            x_low  : lower bound for the option's spot price
            x_up   : upper bound for the option's spot price
        """

        self.rate = rate
        self.sigma = sigma
        self.strike = strike
        self.barrier = barrier
        self.t_up = t_up
        self.x_low = x_low
        self.x_up = x_up

        self.knock = knock
        self.put_call = put_call

    def _is_zero(self, spot):
        """Compute upper boundary conditions for time"""
        if (self.knock == 'up-out' and spot >= self.barrier) or (self.knock == 'down-out' and spot <= self.barrier):
            return True
        elif (self.knock == 'up-in' and spot < self.barrier) or (self.knock == 'down-in' and spot > self.barrier):
            return True
        else: 
            return False
    
    # Returning self boundary conditions:
    def bound_cond_tup(self,  spot):
        if self._is_zero(spot):
            return 0

        if self.put_call == 'call':
            return max(spot - self.strike, 0)
        else:
            return max(self.strike - spot, 0)

    def bound_cond_x_low(self, t):
        if self.put_call == 'call':
            return 0
        else:
            return self.strike * np.exp(-self.rate * (self.tau - t))
    
    def bound_cond_x_up(self, t):
        if self.knock == 'up-out':
            return 0.0
        elif self.knock == 'down-out':
            return self.strike * np.exp(-self.rate * (self.tau - t))
        elif self.knock == 'up-in':
            return self.x_up - self.strike * np.exp(-self.rate * (self.tau - t))
        elif self.knock == 'down-in':
            return 0.0
    

class PDESolver:
    """ Abstract class to solve Black-Scholes PDE """

    def __init__(self, pde, imax, jmax):
        """ Constructor
        Parameters:
            pde : PDE to be solved
            imax : last value of the first variable's discretisation
            jmax : last value of the second variable's discretisation
        """
        # Initialising self variables
        self.pde  = pde
        self.imax = imax
        self.jmax = jmax
        
        # Initialising dt and dx 
        self.dt = pde.t_up / imax
        self.dx = (pde.x_up - pde.x_low) / jmax

        # Initialising grid
        self.grid = np.empty((imax + 1, jmax + 1), dtype = float)

    def t(self, i):
        """ Return the descretised value of t at index i """
        return self.dt * i
    
    def x(self, j):
        """ Return the descretised value of x at index j """
        return self.dx * j

    # Helper umbrella function to get coefficients
    def get_coefficients(self, i, j): return self.pde.get_coefficients(self.t(i), self.x(j))
    
    # Helper umbrella function to get boundary conditions
    def t_up(self, j): return self.pde.bound_cond_tup(self.x(j))
    def x_low(self, i): return self.pde.bound_cond_x_low(self.t(i))
    def x_up(self, i): return self.pde.bound_cond_x_up(self.t(i))


    def interpolate(self, t, x):
        """ Get interpolated solution value at given time and space
        Parameters:
            t : point in time  
            x : point in space
        Return
            interpolated solution value
        """
        # Step 1: calculating closest discrete point (i, j), using floor division "//"
        i = int(t // self.dt) 
        j = int((x - self.pde.x_low) // self.dx)

        # Step 2: calculating vertical and horizontal distance/weight from each node
        i_1 = (t - self.dt * i) / self.dt
        i_0 = 1 - i_1
        j_1 = (x + self.pde.x_low - self.dx * j) / self.dx
        j_0 = 1 - j_1

        # Step 3: Returning the weighted average
        return (i_0 * j_0 * self.grid[i,j] 
                + i_1 * j_0 * self.grid[i+1, j]
                + i_0 * j_1 * self.grid[i, j+1] 
                + i_1 * j_1 * self.grid[i+1, j+1])


class ExplicitScheme(PDESolver):
    """ Black Scholes PDE solver using the explicit scheme
    """
    def __init__(self, pde, imax, jmax):
        super().__init__(pde, imax, jmax)


    # Functions for calculating coefficients
    """ Coefficient {*insert coefficient letter}_{i,j} for explicit scheme
    Parameters:
        i : index of x discretisation
        j : index of t discretisation
    Return:
        Desired coefficient
    """
    def A(self, i, j):
        return (self.dt / self.dx) * ((self.get_coefficients(i, j)['b'] / 2) - (self.get_coefficients(i, j)['a'] / self.dx))
    
    def B(self, i, j):
        return 1 - self.dt * self.get_coefficients(i, j)['c'] + 2 * (self.dt * self.get_coefficients(i, j)['a'] / (self.dx ** 2))
    
    def C(self, i, j):
        return - (self.dt / self.dx) * ((self.get_coefficients(i, j)['b'] / 2) + (self.get_coefficients(i, j)['a'] / self.dx)) 
    
    def D(self, i, j):
        return - self.dt * self.get_coefficients(i, j)['d']
    

    def solve_grid(self):
        """ Method for solving the mesh grid"""
        # 1. Compute all grid points for the last row
        self.grid[self.imax, :] = [self.t_up(j) for j in range(self.jmax + 1)]

        # 2. Iterate for all i from self.imax to 0 inclusive and update grid points:
        for i in range(self.imax, 0, -1):

            # 2.1 (i, 0) boundary conditions for x_low
            self.grid[i - 1, 0] = self.x_low(i - 1)
            # 2.2 (i, -1) boundary conditions for x_up
            self.grid[i - 1, -1] = self.x_up(i - 1)

            # 2.3 v_(i-1, j) formula
            #     Keep in mind, this formula is applied to every row EXCEPT for the
            #     top and bottom row which was already defined by boundary conditions
            self.grid[i-1, 1:self.jmax] = [(self.A(i, j) * self.grid[i, j-1] 
                                            + self.B(i, j) * self.grid[i, j]
                                            + self.C(i, j) * self.grid[i, j+1]
                                            + self.D(i, j)) for j in range(1, self.jmax)]   

In [22]:
# Initialising variables for the pde
spot_init = 100  # Initial Spot price S_0
K         = 100  # Strike price
r         = 0.02 # Risk-free rate
D         = 118  # Barrier
T         = 0.5  # Time to maturity
alpha     = 0.25 # alpha

sigma_const = lambda t, s: 0.1
sigma_1     = lambda t, s: 0.13 * math.exp(-t) * ((100/s)**alpha) # Lambda function representing std

spot_low  = 0
spot_up   = 2 * spot_init

# Initialising up and out pde
pde = Barrier(r, sigma, K, D, T, spot_low, spot_up)

# Executing explicit scheme
explicit = ExplicitScheme(pde, 100, 100)
explicit.solve_grid() # Computing grid

# Interpolating for S = S_0
explicit_val = explicit.interpolate(0,100)
print(f"Explicit Value: {explicit_val}")

Explicit Value: 3.0217292271853795
